# **BLAST 1 version mini**

## *Étape 1 : prétraitement de la query*

La séquence requête (query) est comparée à un dictionnaire de "mots" d'une longueur w, présents dans la banque de données. 
Pour simplifier ici on prendra des séquences d'ADN, le dictionnaire comprendra les 64 codons possibles (w=3). 
L'algorithme va générer une query aléatoire à partir des 4 bases d'ADN, d'une longueur variable, lire tous les "mots" possibles contenus dans cette séquence query qui sont aussi présents dans le dictionnaire.

In [16]:
#création du dictionnaire de codons :
from itertools import product

def generate_codons(alphabet="ATGC", w=3):
    return [''.join(p) for p in product(alphabet, repeat=w)]

dico_codons = generate_codons()
print(dico_codons[:10])  # vérifier

['AAA', 'AAT', 'AAG', 'AAC', 'ATA', 'ATT', 'ATG', 'ATC', 'AGA', 'AGT']


In [17]:
print(dico_codons)
print(len(dico_codons))

['AAA', 'AAT', 'AAG', 'AAC', 'ATA', 'ATT', 'ATG', 'ATC', 'AGA', 'AGT', 'AGG', 'AGC', 'ACA', 'ACT', 'ACG', 'ACC', 'TAA', 'TAT', 'TAG', 'TAC', 'TTA', 'TTT', 'TTG', 'TTC', 'TGA', 'TGT', 'TGG', 'TGC', 'TCA', 'TCT', 'TCG', 'TCC', 'GAA', 'GAT', 'GAG', 'GAC', 'GTA', 'GTT', 'GTG', 'GTC', 'GGA', 'GGT', 'GGG', 'GGC', 'GCA', 'GCT', 'GCG', 'GCC', 'CAA', 'CAT', 'CAG', 'CAC', 'CTA', 'CTT', 'CTG', 'CTC', 'CGA', 'CGT', 'CGG', 'CGC', 'CCA', 'CCT', 'CCG', 'CCC']
64


In [37]:
#création d'une fonction générant aléatoirement une séquence requête (query) :
import random

def random_query(length=10, alphabet="ATGC"): #10 est la longueur de la query générée par défaut si on n'indique pas d'argument autre que 10 quand on appelle la fonction
    return ''.join(random.choice(alphabet) for _ in range(length))

query = random_query(50) #indiquer en argument un nombre suffisamment grand pour pertinence
print(query)

GGACTCGCGCTATAGAAAAATGGGGTCACTGCACCCCGGTTTTCGACCGA


In [38]:
#création de la "banque" de données simplifiée (dictionnaire) :
mini_banque = {
    "Seq1": "ATGCGTACGTA",
    "Seq2": "TTGCGTACCTA",
    "Seq3": "ATCCGTACGGA",           #modifier plus tard
    "Seq4": "GGCATCGTACC",
    "Seq5": "CCTAGCGTATG"
}

# Affichage rapide pour vérifier
for name, seq in mini_banque.items():
    print(name, seq)

Seq1 ATGCGTACGTA
Seq2 TTGCGTACCTA
Seq3 ATCCGTACGGA
Seq4 GGCATCGTACC
Seq5 CCTAGCGTATG


In [39]:
#découpage de la query en k-mers (mots) :
# Paramètres
w = 3  # longueur des mots
T = 2  # score minimal pour qu'un mot soit significatif

def k_mers(sequence, w):
    return [sequence[i:i+w] for i in range(len(sequence)-w+1)]

query_kmers = k_mers(query, w)
print("k-mers de la query :", query_kmers)

k-mers de la query : ['GGA', 'GAC', 'ACT', 'CTC', 'TCG', 'CGC', 'GCG', 'CGC', 'GCT', 'CTA', 'TAT', 'ATA', 'TAG', 'AGA', 'GAA', 'AAA', 'AAA', 'AAA', 'AAT', 'ATG', 'TGG', 'GGG', 'GGG', 'GGT', 'GTC', 'TCA', 'CAC', 'ACT', 'CTG', 'TGC', 'GCA', 'CAC', 'ACC', 'CCC', 'CCC', 'CCG', 'CGG', 'GGT', 'GTT', 'TTT', 'TTT', 'TTC', 'TCG', 'CGA', 'GAC', 'ACC', 'CCG', 'CGA']


In [40]:
#comparaison des k-mers de la query à tous les k-mers du dictionnaire de codons :
#Fonction pour calculer le score entre deux mots
def score_mot(mot1, mot2):
    return sum(1 if a == b else -1 for a, b in zip(mot1, mot2))

In [41]:
#identification des mots significatifs (potentiels hits) :
#Identification des mots significatifs (score >= T)
def mots_significatifs(query_kmers, dico_codons, T):
    resultats = {}
    for km in query_kmers:
        voisins = [mot for mot in dico_codons if score_mot(km, mot) >= T]
        if voisins:
            resultats[km] = voisins
    return resultats

mots_signifs = mots_significatifs(query_kmers, dico_codons, T)
print("Mots significatifs :", mots_signifs)

Mots significatifs : {'GGA': ['GGA'], 'GAC': ['GAC'], 'ACT': ['ACT'], 'CTC': ['CTC'], 'TCG': ['TCG'], 'CGC': ['CGC'], 'GCG': ['GCG'], 'GCT': ['GCT'], 'CTA': ['CTA'], 'TAT': ['TAT'], 'ATA': ['ATA'], 'TAG': ['TAG'], 'AGA': ['AGA'], 'GAA': ['GAA'], 'AAA': ['AAA'], 'AAT': ['AAT'], 'ATG': ['ATG'], 'TGG': ['TGG'], 'GGG': ['GGG'], 'GGT': ['GGT'], 'GTC': ['GTC'], 'TCA': ['TCA'], 'CAC': ['CAC'], 'CTG': ['CTG'], 'TGC': ['TGC'], 'GCA': ['GCA'], 'ACC': ['ACC'], 'CCC': ['CCC'], 'CCG': ['CCG'], 'CGG': ['CGG'], 'GTT': ['GTT'], 'TTT': ['TTT'], 'TTC': ['TTC'], 'CGA': ['CGA']}


In [42]:
#recherche des hits dans la banque :
#Recherche des hits dans la mini banque
hits = {}
for name, seq in mini_banque.items():
    hits_seq = []
    for km in mots_signifs.keys():
        pos = seq.find(km)
        if pos != -1 :
            hits_seq.append((km, pos))
    if hits_seq:
        hits[name] = hits_seq

print("Hits trouvés dans la banque :", hits)

Hits trouvés dans la banque : {'Seq1': [('GCG', 2), ('ATG', 0), ('TGC', 1)], 'Seq2': [('GCG', 2), ('CTA', 8), ('TGC', 1), ('ACC', 6)], 'Seq3': [('GGA', 8), ('CCG', 2), ('CGG', 7)], 'Seq4': [('TCG', 4), ('GCA', 1), ('ACC', 8)], 'Seq5': [('GCG', 4), ('CTA', 1), ('TAT', 7), ('TAG', 2), ('ATG', 8)]}


In [43]:
#Extension sans gap avec debug
def extend_hit(query, seq, q_start, s_start, length, X=2, V=2):
    """
    q_start : position du hit dans la query
    s_start : position du hit dans la séquence
    length  : taille du hit (ici 3)
    X       : tolérance de chute du score
    V       : seuil pour garder un HSP
    """
    best_score = score = sum(1 for i in range(length) if query[q_start+i] == seq[s_start+i])
    best_segment = (q_start, q_start+length, s_start, s_start+length)

    # --- Extension à gauche ---
    i, j = q_start-1, s_start-1
    while i >= 0 and j >= 0:
        score += 1 if query[i] == seq[j] else -1
        if score > best_score:
            best_score = score
            best_segment = (i, best_segment[1], j, best_segment[3])
        if best_score - score >= X:
            break
        i, j = i-1, j-1

    # --- Extension à droite ---
    i, j = best_segment[1], best_segment[3]
    while i < len(query) and j < len(seq):
        score += 1 if query[i] == seq[j] else -1
        if score > best_score:
            best_score = score
            best_segment = (best_segment[0], i+1, best_segment[2], j+1)
        if best_score - score >= X:
            break
        i, j = i+1, j+1

    print(f"Debug: Hit '{query[q_start:q_start+length]}' initial, score max atteint {best_score}")

    if best_score >= V:
        return best_segment, best_score
    else:
        return None, best_score

#Application sur tous les hits
HSPs = {}
for name, seq_hits in hits.items():
    seq = mini_banque[name]
    hsp_list = []
    for km, pos in seq_hits:
        q_pos = query.find(km)
        hsp, score = extend_hit(query, seq, q_pos, pos, len(km))
        if hsp:
            hsp_list.append((hsp, score))
    if hsp_list:
        MSP = max(hsp_list, key=lambda x: x[1])
        HSPs[name] = {"HSPs": hsp_list, "MSP": MSP}

# --- Affichage final ---
print("\nRésultats extension (HSPs et MSPs) :")
for name, data in HSPs.items():
    print(f"{name}: {data}")

Debug: Hit 'GCG' initial, score max atteint 3
Debug: Hit 'ATG' initial, score max atteint 3
Debug: Hit 'TGC' initial, score max atteint 3
Debug: Hit 'GCG' initial, score max atteint 3
Debug: Hit 'CTA' initial, score max atteint 3
Debug: Hit 'TGC' initial, score max atteint 3
Debug: Hit 'ACC' initial, score max atteint 3
Debug: Hit 'GGA' initial, score max atteint 3
Debug: Hit 'CCG' initial, score max atteint 3
Debug: Hit 'CGG' initial, score max atteint 3
Debug: Hit 'TCG' initial, score max atteint 3
Debug: Hit 'GCA' initial, score max atteint 3
Debug: Hit 'ACC' initial, score max atteint 3
Debug: Hit 'GCG' initial, score max atteint 4
Debug: Hit 'CTA' initial, score max atteint 3
Debug: Hit 'TAT' initial, score max atteint 3
Debug: Hit 'TAG' initial, score max atteint 3
Debug: Hit 'ATG' initial, score max atteint 3

Résultats extension (HSPs et MSPs) :
Seq1: {'HSPs': [((6, 9, 2, 5), 3), ((19, 22, 0, 3), 3), ((29, 32, 1, 4), 3)], 'MSP': ((6, 9, 2, 5), 3)}
Seq2: {'HSPs': [((6, 9, 2, 5),